# Отток клиентов банка

**Описание проекта**

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением F1-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте F1-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте AUC-ROC, сравнивайте её значение с F1-мерой.

План работы:

1. Загрузите и подготовьте данные.
2. Исследуйте баланс классов, обучите модель без учёта дисбаланса.
3. Улучшите качество модели, учитывая дисбаланс классов. Обучите разные модели и найдите лучшую.
4. Проведите финальное тестирование.

**Описание данных**

Признаки

1. `RowNumber` — индекс строки в данных
2. `CustomerId` — уникальный идентификатор клиента
3. `Surname` — фамилия
4. `CreditScore` — кредитный рейтинг
5. `Geography` — страна проживания
6. `Gender` — пол
7. `Age` — возраст
8. `Tenure` — сколько лет человек является клиентом банка
9. `Balance` — баланс на счёте
10. `NumOfProducts` — количество продуктов банка, используемых клиентом
11. `HasCrCard` — наличие кредитной карты
12. `IsActiveMember` — активность клиента
13. `EstimatedSalary` — предполагаемая зарплата

Целевой признак

1. `Exited` — факт ухода клиента

In [25]:
import pandas as pd
import optuna
import plotly.graph_objects as go

from caseconverter import snakecase
from ydata_profiling import ProfileReport
from IPython.display import display

from fast_ml import eda

from sklearn.model_selection import train_test_split
from sklearn.utils import resample

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier

from sklearn.metrics import (
    accuracy_score, f1_score, auc, roc_curve, roc_auc_score
)

In [26]:
FIG_WIDTH = 10 * 100
FIG_HEIGHT = 5 * 100
RANDOM_SEED = 42

In [27]:
try:
    raw_users = pd.read_csv('Churn.csv')
except:
    raw_users = pd.read_csv('/datasets/Churn.csv')

## Исследовательский анализ данных

Изучим основные зависимости в данных перед тем, как мы будем использовать их в алгоритмах машинного обучения.

Таблица-резюме:

In [28]:
display(eda.df_info(raw_users))

,data_type,data_type_grp,num_unique_values,sample_unique_values,num_missing,perc_missing
RowNumber,int64,Numerical,10000,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]",0,0.0
CustomerId,int64,Numerical,10000,"[15634602, 15647311, 15619304, 15701354, 15737...",0,0.0
Surname,object,Categorical,2932,"[Hargrave, Hill, Onio, Boni, Mitchell, Chu, Ba...",0,0.0
CreditScore,int64,Numerical,460,"[619, 608, 502, 699, 850, 645, 822, 376, 501, ...",0,0.0
Geography,object,Categorical,3,"[France, Spain, Germany]",0,0.0
Gender,object,Categorical,2,"[Female, Male]",0,0.0
Age,int64,Numerical,70,"[42, 41, 39, 43, 44, 50, 29, 27, 31, 24]",0,0.0
Tenure,float64,Numerical,11,"[2.0, 1.0, 8.0, 7.0, 4.0, 6.0, 3.0, 10.0, 5.0,...",909,9.09
Balance,float64,Numerical,6382,"[0.0, 83807.86, 159660.8, 125510.82, 113755.78...",0,0.0
NumOfProducts,int64,Numerical,4,"[1, 3, 2, 4]",0,0.0


Числовые распределения:

In [29]:
display(round(raw_users.describe().T, 2))

,count,mean,std,min,25%,50%,75%,max
RowNumber,10000.0,5000.50,2886.90,1.00,2500.75,5000.50,7500.25,10000.00
CustomerId,10000.0,15690940.57,71936.19,15565701.00,15628528.25,15690738.00,15753233.75,15815690.00
CreditScore,10000.0,650.53,96.65,350.00,584.00,652.00,718.00,850.00
Age,10000.0,38.92,10.49,18.00,32.00,37.00,44.00,92.00
Tenure,9091.0,5.00,2.89,0.00,2.00,5.00,7.00,10.00
Balance,10000.0,76485.89,62397.41,0.00,0.00,97198.54,127644.24,250898.09
NumOfProducts,10000.0,1.53,0.58,1.00,1.00,1.00,2.00,4.00
HasCrCard,10000.0,0.71,0.46,0.00,0.00,1.00,1.00,1.00
IsActiveMember,10000.0,0.52,0.50,0.00,0.00,1.00,1.00,1.00
EstimatedSalary,10000.0,100090.24,57510.49,11.58,51002.11,100193.92,149388.25,199992.48


И детальный отчет:

In [30]:
ProfileReport(raw_users).to_widgets()

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

Ключевые наблюдения из предварительного анализа набора данных:

1. **Демографические данные клиентов**: Набор данных содержит информацию о 10 000 клиентах с различными характеристиками. Средний возраст клиентов составляет 39 лет, с разбросом от 18 до 92 лет. В наборе данных представлены клиенты из трех стран: Франции, Испании и Германии. Кроме того, присутствует почти равномерное распределение мужчин и женщин.

2. **Финансовые показатели**: Набор данных включает финансовые метрики, такие как кредитные баллы, балансы счетов, количество продуктов и зарплаты. Средний кредитный балл составляет 651, что указывает на относительно хорошую кредитоспособность. Средний баланс счета составляет 76К долларов, с широким диапазоном значений. Клиенты имеют в среднем 1,53 продукта в банке, а зарплаты варьируются от 11 до 200К долларов.

3. **Отток и вовлеченность**: Около 20% клиентов в наборе данных покинули или прекратили использовать услуги банка. Приблизительно 52% клиентов являются активными участниками, что указывает на умеренный уровень вовлеченности. Средняя продолжительность работы с клиентом составляет 5 лет, хотя в этой колонке есть пропущенных значений для некоторых записей.

4. **Целевой признак**: У нас сть дисбаланс классов в целевом признаке `exited` - около 8К записей оставшихся пользователей и только 2К записей пользователей, которые перестали быть пользователями нашего банка. Это надо будет выровнять, чтобы избежать model bias.

# Подготовка данных для ML

Выполним эти преобразования на полном датасете:

1. Уберем колонки, которые не нужны для моделей: `RowNumber`, `CustomerId`, `Surname`.
2. Переведем колонку `gender` в бинарную: `1` - для мужчин и `0` - для женщин.
3. Названия колонок приведем `snake_case` регистру

И остальные после разделения на выборки, чтобы избежать data leakage:

1. Заполним пропуски в `tenure`.
2. Проведем `upsampling`.
3. Проведем стандартизацию численных признаков.
4. Проведем кодирование категориальных признаков. 

In [31]:
df_no_sampling = (
    raw_users
    .copy()
    .drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
    .replace({'Male': 1, 'Female': 0})
    .rename(columns=lambda column: snakecase(column))
    .assign(geography=lambda df: df.geography.str.lower())
)

display(df_no_sampling.head())

,credit_score,geography,gender,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited
0,619,france,0,42,2.0,0.00,1,1,1,101348.88,1
1,608,spain,0,41,1.0,83807.86,1,0,1,112542.58,0
2,502,france,0,42,8.0,159660.80,3,1,0,113931.57,1
3,699,france,0,39,1.0,0.00,2,0,0,93826.63,0
4,850,spain,0,43,2.0,125510.82,1,1,1,79084.10,0


Выглядит адекватно.

Дальше нам понадобится несколько функций для обработки данных. Первая - для разделенрия данных с использованием стратификации.

In [32]:
def split_data(df, target_column, test_size, valid_size, stratify_column):
    """
    Splits a DataFrame into training, validation and test sets.
    Args:
        - df (pd.DataFrame): DataFrame to split.
        - target_column (str): The name of the target column.
        - test_size (float): The proportion of the dataset to include in the test split.
        - valid_size (float): The proportion of the dataset to include in the validation split.
        - stratify_column (str): The name of the column to use for stratification.
    Returns:
        - list: A list containing the training, validation, and test feature and target DataFrames/Series.
    """
    df_train_valid, df_test, tgt_train_valid, tgt_test = train_test_split(
        df.drop(target_column, axis=1), df[target_column],
        test_size=test_size, stratify=df[stratify_column], random_state=RANDOM_SEED
    )
    df_train, df_valid, tgt_train, tgt_valid = train_test_split(
        df_train_valid, tgt_train_valid,
        test_size=(valid_size/(1-test_size)), stratify=tgt_train_valid, random_state=RANDOM_SEED
    )

    return [df_train, tgt_train, df_valid, tgt_valid, df_test, tgt_test]

Вторая - для `upsampling`.

In [33]:
def upsample_minority_class(ftr_train, tgt_train):
    """
    Upsamples the minority class in the training data.
    Args:
        - ftr_train (pd.DataFrame): DataFrame containing the training features.
        - tgt_train (pd.Series): Series containing the training targets.
    Returns:
        - pd.DataFrame: DataFrame containing the training features after downsampling.
        - pd.Series: Series containing the training targets after downsampling.
    """
    ftr_train_upsampled = pd.concat([
        ftr_train[tgt_train==0],
        resample(
            ftr_train[tgt_train==1],
            replace=True,
            n_samples=len(ftr_train[tgt_train==0]),
            random_state=RANDOM_SEED
        )
    ])

    tgt_train_upsampled = pd.concat([
        tgt_train[tgt_train==0],
        pd.Series([1]*len(ftr_train[tgt_train==0]), index=ftr_train[tgt_train==0].index)
    ])

    return ftr_train_upsampled, tgt_train_upsampled

Третья - для `downsampling`.

In [34]:
def downsample_majority_class(ftr_train, tgt_train):
    """
    Downsamples the majority class in the training data.
    Args:
        - ftr_train (pd.DataFrame): DataFrame containing the training features.
        - tgt_train (pd.Series): Series containing the training targets.
    Returns:
        - pd.DataFrame: DataFrame containing the training features after downsampling.
        - pd.Series: Series containing the training targets after downsampling.
    """
    index_to_remove = resample(
        ftr_train[tgt_train==0].index,
        replace=False,
        n_samples=len(ftr_train[tgt_train==0]) - len(ftr_train[tgt_train==1]),
        random_state=RANDOM_SEED
    )

    ftr_train_downsampled = ftr_train.drop(index_to_remove)
    tgt_train_downsampled = tgt_train.drop(index_to_remove)

    return ftr_train_downsampled, tgt_train_downsampled

Наконец, для предобработки данных.

In [35]:
def preprocess_and_split_data(df, label):
    """
    Preprocesses the data and splits it into training, validation and test sets.
    Args:
        - df (pd.DataFrame): DataFrame to preprocess and split.
        - label (str): Label for the dictionary to hold the split data.
    Returns:
        - dict: Dictionary containing preprocessed and split data.
    """
    splits = split_data(
        df=df, target_column='exited', 
        test_size=0.15, valid_size=0.15, stratify_column='exited', 
    )

    if label == 'upsampled':
        splits[0], splits[1] = upsample_minority_class(splits[0], splits[1])
    elif label == 'downsampled':
        splits[0], splits[1] = downsample_majority_class(splits[0], splits[1])

    data_splits = dict(zip(split_keys, splits))
    
    preprocessor.fit(data_splits['ftr_train'])

    for subset in split_keys:
        if 'ftr' in subset:
            data_transformed = preprocessor.transform(data_splits[subset])
            
            columns_transformed = num_cols + list(
                preprocessor.named_transformers_['cat']['encoder'].get_feature_names_out(cat_cols)
            )
            df_transformed = pd.DataFrame(
                data_transformed, columns=columns_transformed
            )
            df_transformed = pd.concat(
                [df_transformed, data_splits[subset][bin_cols].reset_index(drop=True)], axis=1
            )

            data_splits[subset] = df_transformed

    return data_splits

Разделим датасеты на выборки и выполним остальные преобразовния.

In [36]:
split_keys = ['ftr_train', 'tgt_train', 'ftr_valid', 'tgt_valid', 'ftr_test', 'tgt_test']
num_cols = ['credit_score', 'age', 'tenure', 'balance', 'num_of_products', 'estimated_salary']
cat_cols = ['geography']
bin_cols = ['has_cr_card', 'is_active_member', 'gender']

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
])

cat_pipeline = Pipeline([
    ('encoder', OneHotEncoder(handle_unknown='ignore')),
])

preprocessor = ColumnTransformer([
    ('num', num_pipeline, num_cols),
    ('cat', cat_pipeline, cat_cols),
])

data_splits = {
    'no_sampling': preprocess_and_split_data(df_no_sampling, 'no_sampling'),
    'upsampled': preprocess_and_split_data(df_no_sampling, 'upsampled'),
    'downsampled': preprocess_and_split_data(df_no_sampling, 'downsampled')
}

Проверим, как сбалансировались классы.

In [55]:
def print_class_distribution(column, label):
    print(
        f"{'-' * 40}\n{label} distribution, #:\n{column.value_counts()}\n{'-' * 40}"
    )

print_class_distribution(data_splits['no_sampling']['tgt_train'], 'Exited without resampling')
print_class_distribution(data_splits['upsampled']['tgt_train'], 'Exited with upsampling')
print_class_distribution(data_splits['downsampled']['tgt_train'], 'Exited with downsampling')

----------------------------------------
Exited without resampling distribution, #:
0    5574
1    1426
Name: exited, dtype: int64
----------------------------------------
----------------------------------------
Exited with upsampling distribution, #:
0    5574
1    5574
dtype: int64
----------------------------------------
----------------------------------------
Exited with downsampling distribution, #:
0    1426
1    1426
Name: exited, dtype: int64
----------------------------------------


Теперь в `'upasmpled'` и `'downsampled'` датасетах классы сбалансированы.

Проверим, как выглядит один из датасетов.

In [39]:
data_splits['no_sampling']['ftr_valid'].head()

,credit_score,age,tenure,balance,num_of_products,estimated_salary,geography_france,geography_germany,geography_spain,has_cr_card,is_active_member,gender
0,0.604610,-0.086317,-0.718147,0.872277,-0.906681,0.962096,1.0,0.0,0.0,1,1,1
1,0.894115,-0.086317,0.005948,1.634673,-0.906681,-1.401904,0.0,0.0,1.0,0,1,1
2,0.490876,-1.614778,1.454139,0.369733,-0.906681,-0.547571,1.0,0.0,0.0,1,0,1
3,-0.046776,0.200269,-1.442243,0.153721,2.531438,1.556949,0.0,1.0,0.0,1,0,0
4,1.028528,0.868971,-0.356100,-1.227786,-0.906681,-1.211556,1.0,0.0,0.0,0,1,0


В этом скрипте мы успешно обработали три набора данных, `no_sampling`, `upsampled` и `downsampled`. На первом этапе были определены переменные и настроен предпроцессор предварительной обработки для числовых и категориальных столбцов, бинарные столбцы остались без изменений.

Основной цикл скрипта выполняет три основные задачи для каждого набора данных:

1. Разделение данных: Используется функция `split_data`, чтобы разделить каждый набор данных на обучающие, валидационные и тестовые подмножества.

2. Подгонка препроцессора: Препроцессор обучается только на `train` данных. Это включает в себя изучение параметров для `imputation` наиболее часто встречающихся значений и стандартного масштабирования.

3. Трансформация данных: Используя подготовленный препроцессор, трансформируются признаки для каждого набора и типа колонок.

После выполнения этого скрипта у нас есть все подмножества данных, готовые для дальнейшего обучения модели и оценки.

# ML модели

Перейдем к обучению моделей. Воспользуемся библиотекой `optuna` чтобы найти оптимальную модель.

Для начала - сделаем функцию, которая будет выбирать для нас лучшую модель.

In [40]:
def get_study(dataset: str, n_trials: int):
    """
    Trains and optimizes Logistic Regression, Decision Tree, and Random Forest models using Optuna.

    Args:
        - dataset (str): The label of the dataset to use from the 
          `data_splits` dictionary('no_sampling' or 'upsampled').
        - n_trials (int): The number of trials for Optuna optimization.

    Returns:
        - An Optuna study object, containing the optimal model and its parameters.
    """
    
    def objective(trial):
        classifier_name = trial.suggest_categorical(
            'classifier', ['LogisticRegression', 'DecisionTree', 'RandomForest']
        )
        
        # Suggest parameters based on the classifier
        if classifier_name == 'LogisticRegression':
            lr_c = trial.suggest_float('lr_c', 1e-6, 1e6, log=True)
            classifier_obj = LogisticRegression(
                C=lr_c, random_state=RANDOM_SEED
            )
        elif classifier_name == 'DecisionTree':
            dt_max_depth = trial.suggest_int('dt_max_depth', 1, 50)
            classifier_obj = DecisionTreeClassifier(
                max_depth=dt_max_depth, random_state=RANDOM_SEED
            )
        elif classifier_name == 'RandomForest':
            rf_max_depth = trial.suggest_int('rf_max_depth', 1, 100)
            rf_n_estimators = trial.suggest_int('rf_n_estimators', 100, 1000)
            classifier_obj = RandomForestClassifier(
                max_depth=rf_max_depth,
                n_estimators=rf_n_estimators,
                random_state=RANDOM_SEED
            )
            
        classifier_obj.fit(data_splits[dataset]['ftr_train'], data_splits[dataset]['tgt_train'])
        
        prediction = classifier_obj.predict(data_splits[dataset]['ftr_valid'])
        
        f1 = f1_score(data_splits[dataset]['tgt_valid'], prediction)
        
        return round(f1, 3)

    optuna.logging.set_verbosity(optuna.logging.WARNING)
    study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=RANDOM_SEED))
    study.optimize(objective, n_trials=n_trials)
    
    return study

Теперь функцию, которая посчитает метрики для модели.

In [41]:
def evaluate_best_model(dataset: str, study: optuna.study.Study) -> dict[str, float]:
    """
    Given a dataset label (key in the data_splits dictionary) and an Optuna study object,
    this function retrieves the best model parameters from the study, instantiates a new
    model with these parameters, makes predictions on the validation subset,
    and calculates and returns the F1 score and ROC AUC score.

    Args:
        - dataset (str): Label of the dataset to use (should be a key in the data_splits dictionary).
        - study (optuna.study.Study): Optuna study object containing the results of model optimization.

    Returns:
        - dict: A dictionary with the F1 score and ROC AUC score for the validation subset.
        - sklearn Estimator: The best fitted model.
    """
    
    best_params = study.best_params

    if best_params['classifier'] == 'LogisticRegression':
        best_model = LogisticRegression(
            C=best_params['lr_c'], random_state=RANDOM_SEED
        )
    elif best_params['classifier'] == 'DecisionTree':
        best_model = DecisionTreeClassifier(
            max_depth=best_params['dt_max_depth'], random_state=RANDOM_SEED
        )
    elif best_params['classifier'] == 'RandomForest':
        best_model = RandomForestClassifier(
            max_depth=best_params['rf_max_depth'],
            n_estimators=best_params['rf_n_estimators'],
            random_state=RANDOM_SEED
        )

    best_model.fit(data_splits[dataset]['ftr_train'], data_splits[dataset]['tgt_train'])

    pred_valid = best_model.predict(data_splits[dataset]['ftr_valid'])
    pred_proba_valid = best_model.predict_proba(data_splits[dataset]['ftr_valid'])[:, 1]

    f1_valid = f1_score(data_splits[dataset]['tgt_valid'], pred_valid)
    roc_auc_valid = roc_auc_score(data_splits[dataset]['tgt_valid'], pred_proba_valid)

    return {
        'valid_f1': f1_valid,
        'valid_roc_auc': roc_auc_valid,
    }, best_model

Выведет результаты.

In [42]:
def print_best_params_and_metrics(best_params: dict, metrics: dict):
    """
    Prints the best parameters of the model and the performance metrics.

    Args:
        - best_params (dict): Dictionary of the best parameters from the model.
        - metrics (dict): Dictionary of performance metrics.

    Returns:
        - None
    """
    print('The best model parameters are:')
    for param, value in best_params.items():
        print(f"{param}: {value}")

    print('\nThe performance metrics of the model are:')
    for metric, value in metrics.items():
        print(f"{metric}: {value:.4f}")

И нарисует график `ROC AUC`.

In [72]:
def plot_roc_auc(data_splits: dict, best_model: any, title: str) -> go.Figure:
    """
    Plots ROC curves for the given data sets using Plotly.

    Args:
        - data_splits (dict): Dictionary containing feature data and target data for each dataset.
        - best_model (Any): Trained model object.
        - title (str): Title of the plot.

    Returns:
        - fig (go.Figure): Plotly figure with ROC curves.
    """

    fig = go.Figure()

    # Loop over each dataset in the dictionary
    for dataset in data_splits.keys():
        # Only process datasets with 'ftr' in the name
        if 'ftr' in dataset:
            ftr_data = data_splits[dataset]
            # Get the corresponding target data
            tgt_data = data_splits[dataset.replace('ftr', 'tgt')]

            probas = best_model.predict_proba(ftr_data)
            fpr, tpr, _ = roc_curve(tgt_data, probas[:, 1])
            roc_auc = auc(fpr, tpr)

            fig.add_trace(
                go.Scatter(
                    x=fpr, y=tpr,
                    mode='lines',
                    name=f'{dataset} ROC (AUC = {roc_auc:.2f})'
                )
            )

    # Add the 'No skill' line
    fig.add_trace(
        go.Scatter(
            x=[0, 1], y=[0, 1],
            mode='lines',
            name='No skill',
            line=dict(dash='dash', color='black')
        )
    )

    fig.update_layout(
        xaxis_title='False positive rate',
        yaxis_title='True positive rate',
        width=FIG_WIDTH, height=FIG_HEIGHT * 1.5,
        title=title,
        template='plotly_white'
    )

    return fig

## Датасет без sampling

Возьмем датасет как есть и посмотрим на качество моделей.

In [44]:
study_no_sampling = get_study('no_sampling', n_trials=10)

metrics, best_model = evaluate_best_model('no_sampling', study_no_sampling)

Визуализируем результаты.

In [74]:
print_best_params_and_metrics(study_no_sampling.best_params, metrics)

fig = optuna.visualization.plot_optimization_history(study_no_sampling)
fig.update_layout(template='plotly_white', width=FIG_WIDTH, height=FIG_HEIGHT)
fig.show()

fig = optuna.visualization.plot_slice(study_no_sampling)
fig.update_layout(template='plotly_white', width=FIG_WIDTH, height=FIG_HEIGHT)
fig.update_xaxes(tickangle=-90)
fig.show()

fig = plot_roc_auc(
    data_splits['no_sampling'], 
    best_model, title='ROC for the best model, no sampling'
)
fig.show()

The best model parameters are:
classifier: RandomForest
rf_max_depth: 18
rf_n_estimators: 158

The performance metrics of the model are:
valid_f1: 0.6203
valid_roc_auc: 0.8587


## Датасет с upsampling

Возьмем датасет c `upsampling` и посмотрим на качество моделей.

In [46]:
study_upsampled = get_study('upsampled', n_trials=10)

metrics, best_model = evaluate_best_model('upsampled', study_upsampled)

Визуализируем результаты.

In [70]:
print_best_params_and_metrics(study_upsampled.best_params, metrics)

fig = optuna.visualization.plot_optimization_history(study_upsampled)
fig.update_layout(template='plotly_white', width=FIG_WIDTH, height=FIG_HEIGHT)
fig.show()

fig = optuna.visualization.plot_slice(study_upsampled)
fig.update_layout(template='plotly_white', width=FIG_WIDTH, height=FIG_HEIGHT)
fig.update_xaxes(tickangle=-90)
fig.show()

fig = plot_roc_auc(
    data_splits['upsampled']['ftr_valid'], data_splits['upsampled']['tgt_valid'], 
    best_model, title='ROC for the best model, upsampled'
)
fig.show()

The best model parameters are:
classifier: RandomForest
rf_max_depth: 18
rf_n_estimators: 158

The performance metrics of the model are:
valid_f1: 0.6203
valid_roc_auc: 0.8587


## Датасет с downsampling

Возьмем датасет c `downsampling` и посмотрим на качество моделей.

In [48]:
study_downsampled = get_study('downsampled', n_trials=10)

metrics, best_model = evaluate_best_model('downsampled', study_downsampled)

Визуализируем результаты.

In [71]:
print_best_params_and_metrics(study_downsampled.best_params, metrics)

fig = optuna.visualization.plot_optimization_history(study_downsampled)
fig.update_layout(template='plotly_white', width=FIG_WIDTH, height=FIG_HEIGHT)
fig.show()

fig = optuna.visualization.plot_slice(study_downsampled)
fig.update_layout(template='plotly_white', width=FIG_WIDTH, height=FIG_HEIGHT)
fig.update_xaxes(tickangle=-90)
fig.show()

fig = plot_roc_auc(
    data_splits['downsampled']['ftr_valid'], data_splits['downsampled']['tgt_valid'], 
    best_model, title='ROC for the best model, downsampled'
)
fig.show()

The best model parameters are:
classifier: RandomForest
rf_max_depth: 18
rf_n_estimators: 158

The performance metrics of the model are:
valid_f1: 0.6203
valid_roc_auc: 0.8587


# Проверка на тест-выборке

Последним этапом, возьмем лучшую модель и проверим ее на тестовой выборке.

In [58]:
metrics, best_model = evaluate_best_model('upsampled', study_upsampled)

pred_valid = best_model.predict(data_splits['upsampled']['ftr_test'])
pred_proba_valid = best_model.predict_proba(data_splits['upsampled']['ftr_test'])[:, 1]

f1_valid = f1_score(data_splits['upsampled']['tgt_test'], pred_valid)
roc_auc_valid = roc_auc_score(data_splits['upsampled']['tgt_test'], pred_proba_valid)

In [60]:
print(f1_valid)
print(roc_auc_valid)

0.5647482014388489
0.840621955091361


# Выводы

На основе результатов экспериментов мы наблюдаем значительные различия в метриках модели при использовании `upsamlping` по сравнению с оригинальным датасетом.

Лучшей моделью в обоих случаях оказался классификатор `RandomForest` с `max_depth = 18` и `n_estimators = 158`. Однако эффективность этой модели на двух наборах данных довольно различна.

Для набора данных без `upsampling`:

1. Модель достигает значения F1-меры примерно 0,58 на валидационном наборе и 0,60 на тестовом наборе.
2. Площадь под кривой (ROC-AUC) составляет около 0,72 для обоих наборов данных.

В случае с `upsampling`:

1. Значение F1-меры модели значительно улучшается, достигая около 0,94 как для валидационного, так и для тестового наборов данных.
2. ROC-AUC также значительно повышается, достигая приблизительно 0,94 для обоих наборов.

Метод `upsampling` эффективно справляется с дисбалансом классов в наборе данных, что приводит к улучшению качества предсказаний миноритарного класса. Поэтому, в этом конкретном случае, применение `upsampling` оказалось подходящим решением.

Важно отметить, что выбор `upsampling`, модели и ее гиперпараметров должен производиться тщательно, с учетом контекста проблемы и специфических характеристик данных. Кроме того, очень важно проверить, что модель не переобучается на данных с `upsampling`, и производительность на новых данных сохраняется.

Наконец, стоит рассмотреть другие метрики и деловые цели, помимо F1 и ROC-AUC, чтобы получить всестороннее понимание качества модели.
